In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 12 14:07:32 2018

@author: hermet
"""
from scipy.optimize import fsolve
from math import sqrt
#import multiprocessing
import sys
#from numba import jit

def qR():
    qR.u = 138.99118042#137.678405762
    qR.rho = 4.28102874756#2.30249023438
    qR.p = 356908.84375#192077.796875
    qR.t = qR.p/(qR.rho*287.1)
    qR.a = asound(qR.t)
    qR.Mach = Mach(qR.u, qR.a)
    qR.Pi = Pi(qR.p, qR.Mach)
    qR.Ti = Ti(qR.t, qR.Mach)

#@jit
def qL(i, j, k):
    qL.u = 65+ 0.1*i
    qL.rho = 0.01*j + 2.5
    qL.p = 200000 + 10*k
    qL.t = qL.p/(qL.rho*287.1)
    qL.a = asound(qL.t)
    qL.Mach = Mach(qL.u, qL.a)
    qL.Pi = Pi(qL.p, qL.Mach)
    qL.Ti = Ti(qL.t, qL.Mach)


def delta_Ul(p):
    Al = 2/((1.4+1)*qL.rho)
    Bl = ((1.4-1)/(1.4+1))*qL.p  
    if p > qL.p:
        delta = (p-qL.p)*((Al)/(p+Bl))**0.5
    else:
        delta = (2*qL.a)/(1.4-1)*((p/qL.p)**((1.4-1)/(2*1.4))-1)
    return(delta)

   
def delta_Ur(p):
    Ar = 2/((1.4+1)*qR.rho)
    Br = ((1.4-1)/(1.4+1))*qR.p    
    if p > qR.p:
        delta = (p-qR.p)*((Ar)/(p+Br))**0.5
    else:
        delta = (2*qR.a)/(1.4-1)*((p/qR.p)**((1.4-1)/(2*1.4))-1)   
    return(delta)


def isentropic2(pstar):
    rho = qR.rho*(pstar/qR.p)**(1/1.4)      
    return(rho)

def isentropic(pstar):
    rho = qL.rho*(pstar/qL.p)**(1/1.4)      
    return(rho)

def shock_waves(pstar):
    pratio = pstar/qL.p
    gmusgpu = (1.4-1)/(1.4+1)
    rho = qL.rho*(pratio+gmusgpu)/(1+gmusgpu*pratio)      
    return(rho)

def shock_waves2(pstar):
    pratio = pstar/qR.p
    gmusgpu = (1.4-1)/(1.4+1)
    rho = qR.rho*(pratio+gmusgpu)/(1+gmusgpu*pratio)      
    return(rho)


def asound(T):
    return (1.4*287.1*T)**0.5

 
def Mach(u, a):
    return(u/a)

  
def Pi(p, M):
    return(p*(1. + 0.5*(1.4-1)*M**2)**(1.4/(1.4-1)))

def Ti(T, M):
    return(T*(1. + 0.5*(1.4-1)*M**2))

def Ustar_from_pstar(p):
    return(0.5*(qR.u+qL.u+delta_Ur(p)-delta_Ul(p)))

def estimation_pstar():
    return((qL.a + qR.a - 0.5*(1.4-1)*(qR.u - qL.u))/(qL.a/(qL.p)**((1.4-1)/(2*1.4))+qR.a/(qR.p)**((1.4-1)/(2*1.4))))**(2*1.4/(1.4-1))


def solve():
    function = lambda p :delta_Ul(p)+delta_Ur(p)+qR.u-qL.u
    return fsolve(function, estimation_pstar(), xtol = 1e-10)

#@jit
def f():
    for i in range (100):
        #print (i)
        for j in range(100):
            for k in range(100):
                #print (k)
                qL(i, j, k)
                estimation_pstar()
                pstar = solve()
                ustar = Ustar_from_pstar(pstar)
                if pstar > qL.p:
                    rho_Lstar = shock_waves(pstar)
                else:
                    rho_Lstar = isentropic(pstar)
                T_Lstar = pstar/(287.1*rho_Lstar) 
                a_Lstar = asound(T_Lstar)
                Mach_Lstar = Mach(ustar, a_Lstar) 
                Pi_Lstar = Pi(pstar, Mach_Lstar)
                Ti_Lstar = Ti(T_Lstar, Mach_Lstar)
                if abs(Ti_Lstar-Ti_imposed) < Tol1:
                    if abs(Pi_Lstar - Pi_imposed)<Tol2:
                        if pstar > qR.p:
                            rho_Rstar = shock_waves2(pstar)
                            T_Rstar = pstar/(287.1*rho_Rstar) 
                            a_Rstar = asound(T_Rstar)
                            Mach_Rstar = Mach(ustar, a_Rstar) 
                            Pi_Rstar = Pi(pstar, Mach_Rstar)
                            Ti_Rstar = Ti(T_Rstar, Mach_Rstar)
                            print ("I found the solution of the Riemann Problem")
                            print("Choc à droite")
                            print("Rho_rstar", rho_Rstar, "T_rstar", T_Rstar, "Mach_rstar", Mach_Rstar, "Pi_rstar", Pi_Rstar, "Ti_rstar", Ti_Rstar)
                            print("Rho_lstar", rho_Lstar, "T_lstar", T_Lstar, "Mach_lstar", Mach_Lstar, "Pi_lstar", Pi_Lstar, "Ti_lstar", Ti_Lstar)
                            print("pstar", pstar, "ustar", ustar)
                            print("Debit_choc", rho_Rstar*ustar, "Debit_Swave", rho_Lstar*ustar, "Debit_0", qR.rho*qR.u)
                            break
                        else:
                            rho_Rstar = isentropic2(pstar)
                            T_Rstar = pstar/(287.1*rho_Rstar) 
                            a_Rstar = asound(T_Rstar)
                            Mach_Rstar = Mach(ustar, a_Rstar) 
                            Pi_Rstar = Pi(pstar, Mach_Rstar)
                            Ti_Rstar = Ti(T_Rstar, Mach_Rstar)
                            print ("I found the solution of the Riemann Problem")
                            print("Détente à droite")
                            print("Rho_rstar", rho_Rstar, "T_rstar", T_Rstar, "Mach_rstar", Mach_Rstar, "Pi_rstar", Pi_Rstar, "Ti_rstar", Ti_Rstar)
                            print("Rho_lstar", rho_Lstar, "T_lstar", T_Lstar, "Mach_lstar", Mach_Lstar, "Pi_lstar", Pi_Lstar, "Ti_lstar", Ti_Lstar)
                            print("pstar", pstar, "ustar", ustar)
                            print("Debit_choc", rho_Rstar*ustar, "Debit_Swave", rho_Lstar*ustar, "Debit_0", qR.rho*qR.u) 
                            break


#if __name__ == '__main__':
print ('I\'m reading : ', sys.argv[0])
Ti_imposed = 300
Pi_imposed = 300000.0
print ('You want Ti = ',Ti_imposed,'K')
print ('You want Pi = ',Pi_imposed,'Pa')
Tol1 = 10
Tol2 = 100
print ("Your tolerance is : ",Tol1, "K for the Ti and :",Tol2, "Pa for the Pi")
qR()
f()
#pool = multiprocessing.Pool(1)
#pool.map(f, range(1))
#pool.close()
#pool.join()
print('done')

